In [1]:
import numpy as np
import pandas as pd 
from zytholic_project.reviews_data import BaseModelRev


In [ ]:

dfbeer = pd.read_csv("../raw_data/beers_style_renamed.csv")

dftop = pd.read_csv("../raw_data/top_beer_info_style_renamed.csv")

dfrev = pd.read_csv("../raw_data/reviews_top_beer.csv")


In [ ]:
#dfrev.where(dfrev['text'] != '')

dfrev.drop_duplicates(inplace = True)
dfrevclean = dfrev.drop(columns=['username', 'date','Unnamed: 0'])

dfrev['text'] = dfrev['text'].astype(str)
#dfrev['text'] = dfrev['text'].replace('\xa0\xa0',np.nan)
#dfrev['text'].dropna(inplace=True)

#dfrev = dfrev[dfrev['text'] != '\xa0\xa0']

#dfrev.sort_values(by=['beer_id'])


In [ ]:
#dfrev['look'] = dfrev['look'].replace(np.nan,'')
#dfrev2 = dfrev.groupby(['beer_id']).mean()
#dfrev['score'] = dfrev.groupby(['beer_id'])['score'].nanmean()
#dfrev['score'] = dfrev['score'].mean()
#dfrev2.sort_values(by=['beer_id'])
#dfrev['score'].unique()
#dfrev
#dfrevclean

dfrevtext=dfrevclean[['beer_id','text']].copy()


dfrevtext = dfrevtext[dfrev['text'] != '\xa0\xa0']


dfrevtext





In [ ]:
#Regroup and join review by beer_id
#dfrev['score'] = dfrev.groupby(['beer_id'])['score'].transform(lambda x : np.mean(x))
#dfrev['look'] = dfrev.groupby(['beer_id'])['look'].transform(lambda x : np.mean(x))
#dfrev['smell'] = dfrev.groupby(['beer_id'])['smell'].transform(lambda x : np.mean(x))
#dfrev['taste'] = dfrev.groupby(['beer_id'])['taste'].transform(lambda x : np.mean(x))
#dfrev['feel'] = dfrev.groupby(['beer_id'])['feel'].transform(lambda x : np.mean(x))
#dfrev['overall'] = dfrev.groupby(['beer_id'])['overall'].transform(lambda x : np.mean(x))
#dfrev['text'] = dfrev.groupby(['beer_id'])['text'].transform(lambda x : ' '.join(x))

In [ ]:
#dfrev['score','look','smell','taste','feel','overall'] = dfrev.groupby(['beer_id'])['score','look','smell','taste','feel','overall'].transform(lambda x : np.mean(x))
#dfrev['text'] = dfrev.groupby(['beer_id'])['text'].transform(lambda x : ' '.join(x))


In [ ]:
dfrevtext['text'] = dfrev.groupby(['beer_id'])['text'].transform(lambda x : ' '.join(x))
dfrevtext


In [ ]:
#dfrevclean = dfrev.drop(columns=['username', 'date'])
#dfrevclean = dfrevclean[dfrev['text'] != '\xa0\xa0']
#dfrevtext.drop_duplicates(inplace = True)

dfrevtext2 = dfrevtext['text'].unique()



dfrevtext2


In [ ]:
dfrevtext2.head()

In [ ]:
#dfrev = dfrev.groupby(['beer_id']).mean()
#dfrev.drop_duplicates(subset='text',inplace = True)
#dfrev.sort_values(by=['beer_id'])
#dfrevclean = dfrevclean[dfrev['text'] != '\xa0\xa0']

#Cleaning of the reviews Text

dfrevclean['text'] = dfrevclean['text'].str.replace('\xa0','')
import string

def preprocessing(text):
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ') 
    lowercased = text.lower()
    clean = ''.join([i for i in lowercased if not i.isdigit()])
    return clean

dfrevclean['text'] = dfrevclean['text'].apply(preprocessing)


In [ ]:
dfrevclean['text'][0]

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english',min_df=200)

token_pattern = '[a-z]+\w*'


#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(dfrevclean['text'])

#Output the shape of tfidf_matrix

tfidf_matrix.shape

In [ ]:
tfidf.get_feature_names()

In [ ]:
from zytholic_project.reviews_data import BaseModelRev


In [ ]:
model = BaseModelRev()
model.get_data()
model.working_df